In [124]:
# collaboration_visualizer notebook created during the IDIES/UofT Visualization Hackathon - Winter 2019
#
# The idea was to create a simple tool to visualize academic collaborations on a world map,
# by taking author and affiliation lists from scientific papers, matching the authors to their home institutions,
# getting the location (latitude and longitude) of these institutions using geocoder, 
# and showing them on an interactive map - where clicking on the institution marker (sized by number of authors) 
# shows you a list of everyone there, clicking on different collaborations turns those markers on off, etc..
#
# Scraping authors and affialiations from a .pdf was harder than I'd hoped, so for now the author block from the pdf
# needs to copied and pasted to a string authors="""author string""", and the affialiation block from the pdf
# needs to copied and pasted to a string affiliations="""affiliations string""". 
# Given this, I tried to deal with the strings in a way where the authors and institution strings don't need to be 
# modified by hand toooo much... 

# REQUIREMENTS:
# 1.) folium
# 2.) geocoder (or lat long already in a list)

In [125]:
import numpy as np
import os
import re
import folium

In [126]:
from string import digits

def mysplit(s):
    head = s.rstrip('0123456789')
    tail = s[len(head):]
    return [head, tail]

# get list of author names and institutios from large author string
def get_authorlist(stringin, comma):
    """
    takes in a string, returns a list of authors
    and their respective institutions
    stringin: string to split up
    comma: 'leading' or 'trailing' depending on paper author types
                e.g. 'George Stein,1,2' = 'leading'
                    'George Stein1,2' = 'trailing'
    """
    authors  = re.sub('\n', ' ', stringin)
    if comma == 'leading':
        authors  = re.split(r'(?<!\D)[\s]', authors)
    if comma == 'trailing':
        authors  = re.split(r'[,](?!\d)', authors)

    authors = [x for x in authors if x != '']
    auth_name = []
    auth_inst = []
    for i in range(len(authors)):
        author = authors[i].split(',')
        author = [x.strip() for x in author]
        if comma=='trailing':
            if len(author)>1:
                author = mysplit(author[0]) + author[1:]
            else:
                author = mysplit(author[0])
        author = [x for x in author if x != '']
        author[0] = author[0].lstrip()

        auth_name.append(author[0])
        auth_inst.append(author[1:])

    return auth_name, auth_inst
# get list of instutution numbers and names from large institution string

def get_institutions(stringin):
    """
    takes in a string of institutions, returns a list of institutions 
    and a lost of their affiliation numbers
    stringin: string to split up
    """
    institutions = stringin.splitlines()

    inst_name = []
    inst_num  = []
    for i in range(len(institutions)):
        inst_name.append(institutions[i].lstrip(digits))  # get rid of leading numbers
        inst_num.append( int(institutions[i][: len(institutions[i])-len(inst_name[i]) ]) ) # get leading numbers

    inst_name = [x.strip() for x in inst_name]

    return inst_name, inst_num

In [127]:
def author_institution_match(authors_str, comma, institutions_str):
    """
    takes in a string of author names+affiliations, and a string of institutions, 
    returns a list of authors, author institutuions, the institution names, 
    and a list of authors at each institution
    
    authors_str: string of author names and affiliation numbers
    institutions_str: string of institution names and institution numbers
    """
    auth_name, auth_inst = get_authorlist(authors_str, comma)
    inst_name, inst_num  = get_institutions(institutions_str)
              
    inst_auth = ["" for x in range(len(inst_name))]
    num_auth =  np.zeros(len(inst_name)).astype(int)

    #append authors to their home institutions
    for i in range(len(auth_name)):
        for j in range(len(auth_inst[i])):
        
            if inst_auth[int(auth_inst[i][j])-min(inst_num)] == '':
                inst_auth[int(auth_inst[i][j])-min(inst_num)] += auth_name[i]
            else:
                inst_auth[int(auth_inst[i][j])-min(inst_num)] += ', '+auth_name[i]
            num_auth[int(auth_inst[i][j])-min(inst_num)] += 1
    return auth_name, auth_inst, inst_name, inst_num, inst_auth, num_auth

In [128]:
def get_institution_long_lat(inst_name, collaboration):
    """
    takes list of instutution names and returns their longitude and latitude
    inst_name: list of institution names
    collaboration: name of the collaboration to name save_file
    """
    os.environ["GOOGLE_API_KEY"] = "YOUR API KEY HERE"

    import geocoder
    lat_long = np.zeros((len(inst_name), 2))

    for i in range(len(inst_name)):     
        g = geocoder.google(inst_name[i], key= "YOUR API KEY HERE")
        print(g.latlng)
        lat_long[i,:] = g.latlng

    np.savez("data/institution_lat_long_"+collaboration+"_"+str(len(inst_name)), institutions = inst_name, lat_long = lat_long)
    return lat_long

In [129]:
# VARIOUS COLLABORATION AUTHOR LISTS AND AFFILIATION LISTS, COPIED AND PASTED FROM ARXIV PAPERS

In [130]:
simons_auth = """Peter Ade2, James Aguirre3, Zeeshan Ahmed4,5, Simone Aiola6, Aamir Ali7, David Alonso2,8, Marcelo A. Alvarez7,9,
Kam Arnold10, Peter Ashton7,11, Jason Austermann12, Humna Awan13, Carlo Baccigalupi14,15, Taylor Baildon16, Darcy Barron7,17, 
Nick Battaglia18,19, Richard Battye20, Eric Baxter3, Andrew Bazarko6, James A. Beall12, Rachel Bean18, Dominic Beck21,Shawn Beckman7,
Benjamin Beringue22, Federico Bianchini23, Steven Boada13, David Boettger24, J. Richard Bond25, Julian Borrill9,7, Michael L. Brown20,
Sarah Marie Bruno6, Sean Bryan26, Erminia Calabrese2, Victoria Calafut18, Paolo Calisse10,24, Julien Carron27, Anthony Challinor28,22,29, 
Grace Chesmore16, Yuji Chinone7,30, Jens Chluba20, Hsiao-Mei Sherry Cho4, Steve Choi6, Gabriele Coppi3, Nicholas F. Cothard31, 
Kevin Coughlin16, Devin Crichton32, Kevin D. Crowley10, Kevin T. Crowley6, Ari Cukierman7, Mitch D‘Ewart4,
Rolando Dunner24, Tijmen de Haan11,7, Mark Devlin3, Simon Dicker3, Joy Didier33, Matt Dobbs34, Bradley Dober12, 
Cody Duell35, Shannon Duff12, Adri Duivenvoorden36, Jo Dunkley6,37, John Dusatko4, Josquin Errard21, Giulio Fabbian38, Stephen Feeney19, 
Simone Ferraro39, Pedro Fluxa24, Katherine Freese16,36, Josef Frisch4, Andrei Frolov40, George Fuller10, Brittany Fuzia41,
Nicholas Galitzki10, Patricio A. Gallardo35, Jose Tomas Galvez Ghersi40, Jiansong Gao12, Eric Gawiser13,
Martina Gerbino36, Vera Gluscevic42,6,43, Neil Goeckner-Wald7, Joseph Golec16, Sam Gordon44,
Megan Gralla45, Daniel Green10, Arpi Grigorian12, John Groh7, Chris Groppi44, Yilun Guan46,
Jon E. Gudmundsson36, Dongwon Han47, Peter Hargrave2, Masaya Hasegawa48, Matthew Hasselfield49,50,
Makoto Hattori51, Victor Haynes20, Masashi Hazumi48,30, Yizhou He52, Erin Healy6, Shawn Henderson53,5,
Carlos Hervias-Caimapo20, Charles A. Hill7,11, J. Colin Hill19,42, Gene Hilton12, Matt Hilton32,
Adam D. Hincks54,25, Gary Hinshaw55, Renee Hlozek56,57, Shirley Ho11, Shuay-Pwu Patty Ho6, Logan Howe10,
Zhiqi Huang58, Johannes Hubmayr12, Kevin Huffenberger41, John P. Hughes13, Anna Ijjas6,
Margaret Ikape56,57, Kent Irwin4,53,5, Andrew H. Jaffe59, Bhuvnesh Jain3, Oliver Jeong7, Daisuke Kaneko30,
Ethan Karpel53,5, Nobuhiko Katayama30, Brian Keating10, Sarah Kernasovski53,5, Reijo Keskitalo9,7,
Theodore Kisner9,7, Kenji Kiuchi60, Jeff Klein3, Kenda Knowles32, Brian Koopman35, Arthur Kosowsky46,
Nicoletta Krachmalnicoff14, Stephen Kuenstner53,5, Chao-Lin Kuo4,53,5, Akito Kusaka11,60, Jacob Lashner33,
Adrian Lee7,11, Eunseong Lee20, David Leon10, Jason S.-Y. Leung56,57,25, Antony Lewis27, Yaqiong Li6, Zack Li37,
Michele Limon3, Eric Linder11,7, Carlos Lopez-Caraballo24, Thibaut Louis61, Lindsay Lowry10, Marius Lungu6,
Mathew Madhavacheril37, Daisy Mak59, Felipe Maldonado41, Hamdi Mani44, Ben Mates12,
Frederick Matsuda30, Lo¨ıc Maurin24, Phil Mauskopf44, Andrew May20, Nialh McCallum20, Chris McKenney12,
Jeff McMahon16, P. Daniel Meerburg28,22,29,62,63, Joel Meyers25,64, Amber Miller33, Mark Mirmelstein27,
Kavilan Moodley32, Moritz Munchmeyer65, Charles Munson16, Sigurd Naess19, Federico Nati3,
Martin Navaroli10, Laura Newburgh66, Ho Nam Nguyen47, Mike Niemack35, Haruki Nishino48,
John Orlowski-Scherer3, Lyman Page6, Bruce Partridge67, Julien Peloton61,27, Francesca Perrotta14,
Lucio Piccirillo20, Giampaolo Pisano2, Davide Poletti14, Roberto Puddu24, Giuseppe Puglisi53,5, Chris Raum7,
Christian L. Reichardt23, Mathieu Remazeilles20, Yoel Rephaeli68, Dominik Riechers18, Felipe Rojas24,
Anirban Roy14, Sharon Sadeh68, Yuki Sakurai30, Maria Salatino21, Mayuri Sathyanarayana Rao7,11,
Emmanuel Schaan11, Marcel Schmittfull42, Neelima Sehgal47, Joseph Seibert10, Uros Seljak7,11,
Blake Sherwin28,22, Meir Shimon68, Carlos Sierra16, Jonathan Sievers32, Precious Sikhosana32,
Max Silva-Feaver10, Sara M. Simon16, Adrian Sinclair44, Praween Siritanasak10, Kendrick Smith65,
Steve Smith4, David Spergel19,37, Suzanne Staggs6, George Stein25,56, Jason R. Stevens35, Radek Stompor21,
Rashmi Sudiwala2, Aritoki Suzuki11, Osamu Tajima69, Satoru Takakura30, Grant Teply10, Daniel B. Thomas20,
Ben Thorne37,8, Robert Thornton70, Hy Trac52, Calvin Tsai10, Carole Tucker2, Joel Ullom12,
Sunny Vagnozzi36, Alexander van Engelen25, Jeff Van Lanen12, Dan van Winkle53, Eve M. Vavagiakis35,
Clara Verges21, Michael Vissers12, Kasey Wagoner6, Jon Ward3, Ben Westbrook7, Nathan Whitehorn71,
Jason Williams33, Joel Williams20, Edward J. Wollack72, Zhilei Xu3, Jiani Ye47, Byeonghee Yu7,
Cyndria Yu4,5, Fernando Zago46, Hezi Zhang46, Ningfeng Zhu3"""



In [131]:
simons_inst = """2 School of Physics and Astronomy, Cardiff University, The Parade, Cardiff, CF24 3AA, UK
3 Department of Physics and Astronomy, University of Pennsylvania, 209 South 33rd Street, Philadelphia, PA, USA 19104
4 SLAC National Accelerator Laboratory, Menlo Park, CA 94025
5 Kavli Institute for Particle Astrophysics and Cosmology, Menlo Park, CA 94025
6 Joseph Henry Laboratories of Physics, Jadwin Hall, Princeton University, Princeton, NJ, USA 08544
7 Department of Physics, University of California, Berkeley, CA, USA 94720
8 University of Oxford, Denys Wilkinson Building, Keble Road, Oxford OX1 3RH, UK
9 Computational Cosmology Center, Lawrence Berkeley National Laboratory, Berkeley, CA 94720, USA
10 Department of Physics, University of California San Diego, CA, 92093 USA
11 Physics Division, Lawrence Berkeley National Laboratory, Berkeley, CA 94720, USA
12 NIST Quantum Sensors Group, 325 Broadway Mailcode 687.08, Boulder, CO, USA 80305
13 Department of Physics and Astronomy, Rutgers, The State University of New Jersey, Piscataway, NJ USA 08854-8019
14 International School for Advanced Studies (SISSA), Via Bonomea 265, 34136, Trieste, Italy
15 INFN, Sezione di Trieste, Padriciano, 99, 34149 Trieste, Italy
16 Department of Physics, University of Michigan, Ann Arbor, USA 48103
17 Department of Physics and Astronomy, University of New Mexico, Albuquerque, NM 87131, USA
18 Department of Astronomy, Cornell University, Ithaca, NY, USA 14853
19 Center for Computational Astrophysics, Flatiron Institute, 162 5th Avenue, New York, NY 10010, USA
20 Jodrell Bank Centre for Astrophysics, School of Physics and Astronomy, University of Manchester, Manchester, UK
21 AstroParticule et Cosmologie, Univ Paris Diderot, CNRS/IN2P3, CEA/Irfu, Obs de Paris, Sorbonne Paris Cite, France
22 DAMTP, Centre for Mathematical Sciences, University of Cambridge, Wilberforce Road, Cambridge CB3 OWA, UK
23 School of Physics, University of Melbourne, Parkville, VIC 3010, Australia
24 Instituto de Astrofısica and Centro de Astro-Ingenierıa, Facultad de Fısica, Pontificia Universidad Catolica de Chile, Av. Vicuna Mackenna 4860, 7820436 Macul, Santiago, Chile
25 Canadian Institute for Theoretical Astrophysics, University of Toronto, 60 St. George St., Toronto, ON M5S 3H8, Canada
26 School of Electrical, Computer, and Energy Engineering, Arizona State University, Tempe, AZ USA
27 Department of Physics & Astronomy, University of Sussex, Brighton BN1 9QH, UK
28 Kavli Institute for Cosmology Cambridge, Madingley Road, Cambridge CB3 0HA, UK
29 Institute of Astronomy, Madingley Road, Cambridge CB3 0HA, UK
30 Kavli Institute for The Physics and Mathematics of The Universe (WPI), The University of Tokyo, Kashiwa, 277- 8583, Japan
31 Department of Applied and Engineering Physics, Cornell University, Ithaca, NY, USA 14853
32 Astrophysics and Cosmology Research Unit, School of Mathematics, Statistics and Computer Science, University of KwaZulu-Natal, Durban 4041, South Africa
33 University of Southern California. Department of Physics and Astronomy, 825 Bloom Walk ACB 439, Los Angeles, CA 90089-0484
34 Physics Department, McGill University, Montreal, QC H3A 0G4, Canada
35 Department of Physics, Cornell University, Ithaca, NY, USA 14853
36 The Oskar Klein Centre for Cosmoparticle Physics, Department of Physics, Stockholm University, AlbaNova, SE-106 91 Stockholm, Sweden
37 Department of Astrophysical Sciences, Peyton Hall, Princeton University, Princeton, NJ, USA 08544
38 Institut d’Astrophysique Spatiale, CNRS (UMR 8617), Univ. Paris-Sud, Universite Paris-Saclay, Bˆat. 121, 91405 Orsay, France.
39 Berkeley Center for Cosmological Physics, University of California, Berkeley, CA 94720, USA
40 Simon Fraser University, 8888 University Dr, Burnaby, BC V5A 1S6, Canada
41 Department of Physics, Florida State University, Tallahassee FL, USA 32306
42 Institute for Advanced Study, 1 Einstein Dr, Princeton, NJ 08540
43 Department of Physics, University of Florida, Gainesville, Florida 32611, USA
44 School of Earth and Space Exploration, Arizona State University, Tempe, AZ, USA 85287
45 University of Arizona, 933 N Cherry Ave, Tucson, AZ 85719
46 Department of Physics and Astronomy, University of Pittsburgh, Pittsburgh, PA, USA 15260
47 Physics and Astronomy Department, Stony Brook University, Stony Brook, NY USA 11794
48 High Energy Accelerator Research Organization (KEK), Tsukuba, Ibaraki 305-0801, Japan
49 Department of Astronomy and Astrophysics, The Pennsylvania State University, University Park, PA 16802
50 Institute for Gravitation and the Cosmos, The Pennsylvania State University, University Park, PA 16802
51 Astronomical Institute, Graduate School of Science, Tohoku University, Sendai, 980-8578, Japan
52 McWilliams Center for Cosmology, Department of Physics, Carnegie Mellon University, 5000 Forbes Avenue, Pittsburgh, PA 15213, USA
53 Department of Physics, Stanford University, Stanford, CA 94305
54 Department of Physics, University of Rome “La Sapienza”, Piazzale Aldo Moro 5, I-00185 Rome, Italy
55 Department of Physics and Astronomy, University of British Columbia, Vancouver, BC, Canada V6T 1Z1
56 Department of Astronomy and Astrophysics, University of Toronto, 50 St. George St., Toronto, ON M5S 3H4, Canada
57 Dunlap Institute for Astronomy and Astrophysics, University of Toronto, 50 St. George St., Toronto, ON M5S 3H4, Canada
58 School of Physics and Astronomy, Sun Yat-Sen University, 135 Xingang Xi Road, Guangzhou, China
59 Imperial College London, Blackett Laboratory, SW7 2AZ UK
60 Department of Physics, University of Tokyo, Tokyo 113-0033, Japan
61 Laboratoire de l’Accelerateur Lineaire, Univ. Paris-Sud, CNRS/IN2P3, Universite Paris-Saclay, Orsay, France
62 Van Swinderen Institute for Particle Physics and Gravity, University of Groningen, Nijenborgh 4, 9747 AG Groningen, The Netherlands
63 Kapteyn Astronomical Institute, University of Groningen, P.O. Box 800, 9700 AV Groningen, The Netherlands
64 Dedman College of Humanities and Sciences, Department of Physics, Southern Methodist University, 3215 Daniel Ave. Dallas, Texas 75275-0175
65 Perimeter Institute 31 Caroline Street North, Waterloo, Ontario, Canada, N2L 2Y5
66 Department of Physics, Yale University, New Haven, CT 06520, USA
67 Department of Physics and Astronomy, Haverford College,Haverford, PA, USA 19041
68 Raymond and Beverly Sackler School of Physics and Astronomy, Tel Aviv University, P.O. Box 39040, Tel Aviv 6997801, Israel
69 Department of Physics, Faculty of Science, Kyoto University, Kyoto 606, Japan
70 Department of Physics and Engineering, 720 S. Church St., West Chester, PA 19383
71 Department of Physics and Astronomy, University of California Los Angeles, 475 Portola Plaza, Los Angeles, CA 9009
72 NASA/Goddard Space Flight Center, Greenbelt, MD, USA 20771"""

In [132]:
planck_auth = """P. A. R. Ade93, N. Aghanim65, C. Armitage-Caplan99, M. Arnaud79, M. Ashdown76,6
, F. Atrio-Barandela19, J. Aumont65
,
C. Baccigalupi92, A. J. Banday102,10, R. B. Barreiro72, J. G. Bartlett1,74, E. Battaner105, K. Benabed66,101, A. Benoit63, A. Benoit-Levy
26,66,101
,
J.-P. Bernard102,10, M. Bersanelli38,55, P. Bielewicz102,10,92, J. Bobin79, J. J. Bock74,11, A. Bonaldi75, J. R. Bond9
, J. Borrill14,96, F. R. Bouchet66,101
,
M. Bridges76,6,69, M. Bucher1
, C. Burigana54,36, R. C. Butler54, E. Calabrese99, B. Cappellini55, J.-F. Cardoso80,1,66, A. Catalano81,78
,
A. Challinor69,76,12, A. Chamballu79,16,65, R.-R. Chary62, X. Chen62, H. C. Chiang30,7
, L.-Y Chiang68, P. R. Christensen88,41, S. Church98
,
D. L. Clements61, S. Colombi66,101, L. P. L. Colombo25,74, F. Couchot77, A. Coulais78, B. P. Crill74,89, A. Curto6,72, F. Cuttaia54, L. Danese92
,
R. D. Davies75, R. J. Davis75, P. de Bernardis37, A. de Rosa54, G. de Zotti50,92, J. Delabrouille1
, J.-M. Delouis66,101, F.-X. Desert
58, C. Dickinson75
,
J. M. Diego72, K. Dolag104,84, H. Dole65,64, S. Donzelli55, O. Dore
74,11, M. Douspis65, J. Dunkley99, X. Dupac44, G. Efstathiou69
, F. Elsner66,101
,
T. A. Enblin84, H. K. Eriksen70, F. Finelli54,56, O. Forni102,10, M. Frailis52, A. A. Fraisse30, E. Franceschi54, T. C. Gaier74, S. Galeotta52, S. Galli66
,
K. Ganga1
, M. Giard102,10, G. Giardino45, Y. Giraud-Heraud
1
, E. Gjerlow70, J. Gonzalez-Nuevo
72,92, K. M. Gorski
74,106, S. Gratton76,69
,
A. Gregorio39,52, A. Gruppuso54, J. E. Gudmundsson30, J. Haissinski77, J. Hamann100, F. K. Hansen70, D. Hanson85,74,9
, D. Harrison69,76
,
S. Henrot-Versille
77, C. Hernandez-Monteagudo
13,84, D. Herranz72, S. R. Hildebrandt11, E. Hivon66,101, M. Hobson6
, W. A. Holmes74
,
A. Hornstrup17, Z. Hou32, W. Hovest84, K. M. Huffenberger28, A. H. Jaffe
61, T. R. Jaffe
102,10, J. Jewell74, W. C. Jones30, M. Juvela29
,
E. Keihanen
29, R. Keskitalo23,14, T. S. Kisner83, R. Kneissl43,8
, J. Knoche84, L. Knox32, M. Kunz18,65,3
, H. Kurki-Suonio29,48, G. Lagache65
,
A. Lahteenmaki
2,48, J.-M. Lamarre78, A. Lasenby6,76, M. Lattanzi36, R. J. Laureijs45, C. R. Lawrence74, S. Leach92, J. P. Leahy75, R. Leonardi44
,
J. Leon-Tavares
46,2
, J. Lesgourgues100,91, A. Lewis27, M. Liguori35, P. B. Lilje70, M. Linden-Vørnle17, M. Lopez-Caniego
72, P. M. Lubin33
,
J. F. Macıas-Perez
81, B. Maffei75, D. Maino38,55, N. Mandolesi54,5,36, M. Maris52, D. J. Marshall79, P. G. Martin9
, E. Martinez-Gonzalez72
,
S. Masi37, M. Massardi53, S. Matarrese35, F. Matthai84, P. Mazzotta40, P. R. Meinhold33, A. Melchiorri37,57, J.-B. Melin16, L. Mendes44
,
E. Menegoni37, A. Mennella38,55, M. Migliaccio69,76, M. Millea32, S. Mitra60,74, M.-A. Miville-Deschenes
65,9
, A. Moneti66, L. Montier102,10
,
G. Morgante54, D. Mortlock61, A. Moss94, D. Munshi93, J. A. Murphy87, P. Naselsky88,41, F. Nati37, P. Natoli36,4,54, C. B. Netterfield21
,
H. U. Norgaard-Nielsen17, F. Noviello75, D. Novikov61, I. Novikov88, I. J. O’Dwyer74, S. Osborne98, C. A. Oxborrow17, F. Paci92, L. Pagano37,57
,
F. Pajot65, D. Paoletti54,56, B. Partridge47, F. Pasian52, G. Patanchon1
, D. Pearson74, T. J. Pearson11,62, H. V. Peiris26, O. Perdereau77, L. Perotto81
,
F. Perrotta92, V. Pettorino18, F. Piacentini37, M. Piat1
, E. Pierpaoli25, D. Pietrobon74, S. Plaszczynski77, P. Platania73, E. Pointecouteau102,10
,
G. Polenta4,51, N. Ponthieu65,58, L. Popa67, T. Poutanen48,29,2
, G. W. Pratt79, G. Prezeau
11,74, S. Prunet66,101, J.-L. Puget65, J. P. Rachen22,84
,
W. T. Reach103, R. Rebolo71,15,42, M. Reinecke84, M. Remazeilles75,65,1
, C. Renault81, S. Ricciardi54, T. Riller84, I. Ristorcelli102,10, G. Rocha74,11
,
C. Rosset1
, G. Roudier1,78,74, M. Rowan-Robinson61, J. A. Rubino-Martin71,42, B. Rusholme62, M. Sandri54, D. Santos81, M. Savelainen29,48
,
G. Savini90, D. Scott24, M. D. Seiffert74,11, E. P. S. Shellard12, L. D. Spencer93, J.-L. Starck79, V. Stolyarov6,76,97, R. Stompor1
, R. Sudiwala93
,
R. Sunyaev84,95, F. Sureau79, D. Sutton69,76, A.-S. Suur-Uski29,48, J.-F. Sygnet66, J. A. Tauber45, D. Tavagnacco52,39, L. Terenzi54, L. Toffolatti20,72
,
M. Tomasi55, M. Tristram77, M. Tucci18,77, J. Tuovinen86, M. Turler
59, G. Umana49, L. Valenziano54, J. Valiviita48,29,70, B. Van Tent82, P. Vielva72
,
F. Villa54, N. Vittorio40, L. A. Wade74, B. D. Wandelt66,101,34, I. K. Wehus74, M. White31, S. D. M. White84, A. Wilkinson75, D. Yvon16
,
A. Zacchei52, A. Zonca33"""
planck_auth = re.sub('\n', ' ', planck_auth)


In [133]:
planck_inst = """1 APC, AstroParticule et Cosmologie, Universite Paris Diderot, CNRS/IN2P3, CEA/lrfu, Observatoire de Paris, Sorbonne Paris Cite, 10, rue Alice Domon et Leonie Duquet, 75205 Paris Cedex 13, France
2 Aalto University Metsahovi Radio Observatory, Metsahovintie 114, FIN-02540 Kylmala, Finland
3 African Institute for Mathematical Sciences, 6-8 Melrose Road, Muizenberg, Cape Town, South Africa
4 Agenzia Spaziale Italiana Science Data Center, Via del Politecnico snc, 00133, Roma, Italy
5 Agenzia Spaziale Italiana, Viale Liegi 26, Roma, Italy
6 Astrophysics Group, Cavendish Laboratory, University of Cambridge, J J Thomson Avenue, Cambridge CB3 0HE, U.K.
7 Astrophysics & Cosmology Research Unit, School of Mathematics, Statistics & Computer Science, University of KwaZulu-Natal, Westville Campus, Private Bag X54001, Durban 4000, South Africa
8 Atacama Large Millimeter/submillimeter Array, ALMA Santiago Central Offices, Alonso de Cordova 3107, Vitacura, Casilla 763 0355, Santiago, Chile
9 CITA, University of Toronto, 60 St. George St., Toronto, ON M5S 3H8, Canada
10 CNRS, IRAP, 9 Av. colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France
11 California Institute of Technology, Pasadena, California, U.S.A.
12 Centre for Theoretical Cosmology, DAMTP, University of Cambridge, Wilberforce Road, Cambridge CB3 0WA, U.K.
13 Centro de Estudios de Fisica del Cosmos de Aragon (CEFCA), Plaza San Juan, 1, planta 2, E-44001, Teruel, Spain
14 Computational Cosmology Center, Lawrence Berkeley National Laboratory, Berkeley, California, U.S.A.
15 Consejo Superior de Investigaciones Cientificas (CSIC), Madrid, Spain
16 DSM/Irfu/SPP, CEA-Saclay, F-91191 Gif-sur-Yvette Cedex, France
17 DTU Space, National Space Institute, Technical University of Denmark, Elektrovej 327, DK-2800 Kgs. Lyngby, Denmark
18 Departement de Physique Theorique, Universite de Geneve, 24, Quai E. Ansermet,1211 Geneve 4, Switzerland
19 Departamento de Fisica Fundamental, Facultad de Ciencias, Universidad de Salamanca, 37008 Salamanca, Spain
20 Departamento de Fisica, Universidad de Oviedo, Avda. Calvo Sotelo s/n, Oviedo, Spain
21 Department of Astronomy and Astrophysics, University of Toronto, 50 Saint George Street, Toronto, Ontario, Canada
22 Department of Astrophysics/IMAPP, Radboud University Nijmegen, P.O. Box 9010, 6500 GL Nijmegen, The Netherlands
23 Department of Electrical Engineering and Computer Sciences, University of California, Berkeley, California, U.S.A.
24 Department of Physics & Astronomy, University of British Columbia, 6224 Agricultural Road, Vancouver, British Columbia, Canada
25 Department of Physics and Astronomy, Dana and David Dornsife College of Letter, Arts and Sciences, University of Southern California, Los Angeles, CA 90089, U.S.A.
26 Department of Physics and Astronomy, University College London, London WC1E 6BT, U.K.
27 Department of Physics and Astronomy, University of Sussex, Brighton BN1 9QH, U.K.
28 Department of Physics, Florida State University, Keen Physics Building, 77 Chieftan Way, Tallahassee, Florida, U.S.A.
29 Department of Physics, Gustaf Hallstromin katu 2a, University of Helsinki, Helsinki, Finland
30 Department of Physics, Princeton University, Princeton, New Jersey, U.S.A.
31 Department of Physics, University of California, Berkeley, California, U.S.A.
32 Department of Physics, University of California, One Shields Avenue, Davis, California, U.S.A.
33 Department of Physics, University of California, Santa Barbara, California, U.S.A.
34 Department of Physics, University of Illinois at Urbana-Champaign, 1110 West Green Street, Urbana, Illinois, U.S.A.
35 Dipartimento di Fisica e Astronomia G. Galilei, Universita degli Studi di Padova, via Marzolo 8, 35131 Padova, Italy
36 Dipartimento di Fisica e Scienze della Terra, Universita di Ferrara, Via Saragat 1, 44122 Ferrara, Italy
37 Dipartimento di Fisica, Universita La Sapienza, P. le A. Moro 2, Roma, Italy
38 Dipartimento di Fisica, Universita degli Studi di Milano, Via Celoria, 16, Milano, Italy
39 Dipartimento di Fisica, Universita degli Studi di Trieste, via A. Valerio 2, Trieste, Italy
40 Dipartimento di Fisica, Universita di Roma Tor Vergata, Via della Ricerca Scientifica, 1, Roma, Italy
41 Discovery Center, Niels Bohr Institute, Blegdamsvej 17, Copenhagen, Denmark
42 Dpto. Astrofisica, Universidad de La Laguna (ULL), E-38206 La Laguna, Tenerife, Spain
43 European Southern Observatory, ESO Vitacura, Alonso de Cordova 3107, Vitacura, Casilla 19001, Santiago, Chile
44 European Space Agency, ESAC, Planck Science Office, Camino bajo del Castillo, s/n, Urbanizacion Villafranca del Castillo, Villanueva de la Canada, Madrid, Spain
45 European Space Agency, ESTEC, Keplerlaan 1, 2201 AZ Noordwijk, The Netherlands
46 Finnish Centre for Astronomy with ESO (FINCA), University of Turku, Vaisalantie 20, FIN-21500, Piikkio, Finland
47 Haverford College Astronomy Department, 370 Lancaster Avenue, Haverford, Pennsylvania, U.S.A.
48 Helsinki Institute of Physics, Gustaf Hallstromin katu 2, University of Helsinki, Helsinki, Finland
49 INAF - Osservatorio Astrofisico di Catania, Via S. Sofia 78, Catania, Italy
50 INAF - Osservatorio Astronomico di Padova, Vicolo dell’Osservatorio 5, Padova, Italy
51 INAF - Osservatorio Astronomico di Roma, via di Frascati 33, Monte Porzio Catone, Italy
52 INAF - Osservatorio Astronomico di Trieste, Via G.B. Tiepolo 11, Trieste, Italy
53 INAF Istituto di Radioastronomia, Via P. Gobetti 101, 40129 Bologna, Italy
54 INAF/IASF Bologna, Via Gobetti 101, Bologna, Italy
55 INAF/IASF Milano, Via E. Bassini 15, Milano, Italy
56 INFN, Sezione di Bologna, Via Irnerio 46, I-40126, Bologna, Italy
57 INFN, Sezione di Roma 1, Universita di Roma Sapienza, Piazzale Aldo Moro 2, 00185, Roma, Italy
58 IPAG: Institut de Planetologie et d’Astrophysique de Grenoble, Universite Joseph Fourier, Grenoble 1 CNRS-INSU, UMR 5274, Grenoble, F-38041, France
59 ISDC Data Centre for Astrophysics, University of Geneva, ch. d’Ecogia 16, Versoix, Switzerland
60 IUCAA, Post Bag 4, Ganeshkhind, Pune University Campus, Pune 411 007, India
61 Imperial College London, Astrophysics group, Blackett Laboratory, Prince Consort Road, London, SW7 2AZ, U.K.
62 Infrared Processing and Analysis Center, California Institute of Technology, Pasadena, CA 91125, U.S.A.
63 Institut Neel, CNRS, Universite Joseph Fourier Grenoble I, 25 rue des Martyrs, Grenoble, France
64 Institut Universitaire de France, 103, bd Saint-Michel, 75005, Paris, France
65 Institut d’Astrophysique Spatiale, CNRS (UMR8617) Universite Paris-Sud 11, Batiment 121, Orsay, France
66 Institut d’Astrophysique de Paris, CNRS (UMR7095), 98 bis Boulevard Arago, F-75014, Paris, France
67 Institute for Space Sciences, Bucharest-Magurale, Romania
68 Institute of Astronomy and Astrophysics, Academia Sinica, Taipei, Taiwan
69 Institute of Astronomy, University of Cambridge, Madingley Road, Cambridge CB3 0HA, U.K.
70 Institute of Theoretical Astrophysics, University of Oslo, Blindern, Oslo, Norway
71 Instituto de Astrofisica de Canarias, C/Via Lactea s, La Laguna,Tenerife, Spain
72 Instituto de Fisica de Cantabria (CSIC-Universidad de Cantabria), Avda. de los Castros s/n, Santander, Spain
73 Istituto di Fisica del Plasma, CNR-ENEA-EURATOM Association, Via R. Cozzi 53, Milano, Italy
74 Jet Propulsion Laboratory, California Institute of Technology, 4800 Oak Grove Drive, Pasadena, California, U.S.A.
75 Jodrell Bank Centre for Astrophysics, Alan Turing Building, School of Physics and Astronomy, The University of Manchester, Oxford Road, Manchester, M13 9PL, U.K.
76 Kavli Institute for Cosmology Cambridge, Madingley Road, Cambridge, CB3 0HA, U.K.
77 LAL, Universite Paris-Sud, CNRS /IN2P3, Orsay, France
78 LERMA, CNRS, Observatoire de Paris, 61 Avenue del’Observatoire, Paris, France
79 Laboratoire AIM, IRFU/Service d’Astrophysique - CEA/DSM - CNRS - Universite Paris Diderot, Bat. 709, CEA-Saclay, F-91191 Gif-sur-Yvette Cedex, France
80 Laboratoire Traitement et Communication de l’Information, CNRS (UMR 5141) and Telecom ParisTech, 46 rue Barrault F-75634 Paris Cedex 13, France
81 Laboratoire de Physique Subatomique et de Cosmologie, Universite Joseph Fourier Grenoble I, CNRS IN2P3, Institut National Polytechnique de Grenoble, 53 rue des Martyrs, 38026 Grenoble cedex, France
82 Laboratoire de Physique Theorique, Universite Paris-Sud 11 & CNRS, Batiment 210, 91405 Orsay, France
83 Lawrence Berkeley National Laboratory, Berkeley, California, U.S.A.
84 Max-Planck-Institut fur Astrophysik, Karl-Schwarzschild-Str. 1, 85741 Garching, Germany
85 McGill Physics, Ernest Rutherford Physics Building, McGill University, 3600 rue University, Montreal, QC, H3A 2T8, Canada
86 MilliLab, VTT Technical Research Centre of Finland, Tietotie 3, Espoo, Finland
87 National University of Ireland, Department of Experimental Physics, Maynooth, Co. Kildare, Ireland
88 Niels Bohr Institute, Blegdamsvej 17, Copenhagen, Denmark
89 Observational Cosmology, Mail Stop 367-17, California Institute of Technology, Pasadena, CA, 91125, U.S.A.
90 Optical Science Laboratory, University College London, Gower Street, London, U.K.
91 SB-ITP-LPPC, EPFL, CH-1015, Lausanne, Switzerland
92 SISSA, Astrophysics Sector, via Bonomea 265, 34136, Trieste, Italy
93 School of Physics and Astronomy, Cardiff University, Queens Buildings, The Parade, Cardiff, CF24 3AA, U.K.
94 School of Physics and Astronomy, University of Nottingham, Nottingham NG7 2RD, U.K.
95 Space Research Institute (IKI), Russian Academy of Sciences, Profsoyuznaya Str, 84/32, Moscow, 117997, Russia
96 Space Sciences Laboratory, University of California, Berkeley, California, U.S.A.
97 Special Astrophysical Observatory, Russian Academy of Sciences, Nizhnij Arkhyz, Zelenchukskiy region, Karachai-Cherkessian Republic, 369167, Russia
98 Stanford University, Dept of Physics, Varian Physics Bldg, 382 Via Pueblo Mall, Stanford, California, U.S.A.
99 Sub-Department of Astrophysics, University of Oxford, Keble Road, Oxford OX1 3RH, U.K.
100 Theory Division, PH-TH, CERN, CH-1211, Geneva 23, Switzerland
101 UPMC Univ Paris 06, UMR7095, 98 bis Boulevard Arago, F-75014, Paris, France
102 Universite de Toulouse, UPS-OMP, IRAP, F-31028 Toulouse cedex 4, France
103 Universities Space Research Association, Stratospheric Observatory for Infrared Astronomy, MS 232-11, Moffett Field, CA 94035, U.S.A.
104 University Observatory, Ludwig Maximilian University of Munich, Scheinerstrasse 1, 81679 Munich, Germany
105 University of Granada, Departamento de Fisica Teorica y del Cosmos, Facultad de Ciencias, Granada, Spain
106 Warsaw University Observatory, Aleje Ujazdowskie 4, 00-478 Warszawa, Poland"""

In [134]:
#WMAP 9 year
wmap_auth="""G. Hinshaw1, D. Larson2, E. Komatsu3,4,5, D. N. Spergel6,4, C. L. Bennett2, J. Dunkley7, M. R. Nolta8, 
M. Halpern1, R. S. Hill9, N. Odegard9, L. Page10, K. M. Smith6,11, J. L. Weiland2, B. Gold12, N. Jarosik10, A.Kogut13, 
M. Limon14, S. S. Meyer15, G. S. Tucker16, E. Wollack13, E. L. Wright17"""

wmap_inst="""1 Dept. of Physics and Astronomy, University of British Columbia, Vancouver, BC Canada V6T 1Z1
2 Dept. of Physics & Astronomy, The Johns Hopkins University, 3400 N. Charles St., Baltimore, MD 21218-2686
3 Max-Planck-Institut f¨ur Astrophysik, Karl-Schwarzschild Str. 1, 85741 Garching, Germany
4 Kavli Institute for the Physics and Mathematics of the Universe, Todai Institutes for Advanced Study, the University of Tokyo, Kashiwa, Japan 277-8583 (Kavli IPMU, WPI)
5 Texas Cosmology Center and Dept. of Astronomy, Univ. of Texas, Austin, 2511 Speedway, RLM 15.306, Austin, TX 78712
6 Dept. of Astrophysical Sciences, Peyton Hall, Princeton University, Princeton, NJ 08544-1001
7 Oxford Astrophysics, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK
8 Canadian Institute for Theoretical Astrophysics, 60 St. George St, University of Toronto, Toronto, ON Canada M5S 3H8
9 ADNET Systems, Inc., 7515 Mission Dr., Suite A100 Lanham, Maryland 20706
10 Dept. of Physics, Jadwin Hall, Princeton University, Princeton, NJ 08544-0708
11 Perimeter Institute for Theoretical Physics, Waterloo, ON N2L 2Y5, Canada
12 University of Minnesota, School of Physics & Astronomy, 116 Church Street S.E., Minneapolis, MN 55455
13 Code 665, NASA/Goddard Space Flight Center, Greenbelt, MD 20771
14 Columbia Astrophysics Laboratory, 550 W. 120th St., Mail Code 5247, New York, NY 10027-6902
15 Depts. of Astrophysics and Physics, KICP and EFI, University of Chicago, Chicago, IL 60637
16 Dept. of Physics, Brown University, 182 Hope St., Providence, RI 02912-1843
17 UCLA Physics & Astronomy, PO Box 951547, Los Angeles, CA 90095–1547"""

In [135]:
#COBE
cobe_auth="""G. F. Smoot1, C. L. Bennett2, A. Kogut3, E. L. Wright4, J. Aymon1, N. W. Boggess2, E. S. Cheng2, G. de Amici1,
S. Gulkis5, M. G. Hauser2, G. Hinshaw3, P. D. Jackson6, M. Janssen5, E. Kaita6, T. Kelsall2, P. Keegstra6, C. Lineweaver1, K. Loewenstein6,
P. Lubin7, J. Mather2, S. S. Meyer8, S. H. Moseley2, T. Murdock9, L. Rokke6, R. F. Silverberg2, L. Tenorio1, R. Weiss8, D. T. Wilkinson10"""

cobe_inst="""1 Lawrence Berkeley Laboratory; California, University, Berkeley 
2 NASA, Goddard Space Flight Center, Greenbelt, MD 
3 University Space Research Association; NASA, Goddard Space Flight Center, Greenbelt, MD 20771
4 University of California at Los Angeles, Astronomy Department, Los Angeles, CA 90024-1562 
5 Jet Propulsion Laboratory, Pasadena, CA 91109
6 Hughes STX Corporation, 4400 Forbes Boulevard, Lanham, MD 20706
7 University of California at Santa Barbara, Physics Department, Santa Barbara, CA 93106
8 Massachusetts Institute of Technology, Department of Physics, Cambridge, MA 02139
9 General Research Corporation, 5 Cherry Hill Drive, Danvers, MA 01923
10 Princeton University, Department of Physics, Princeton, NJ 08504"""

In [136]:
# GET DATA FOR MAP

In [137]:
comma = 'trailing'
# Simons
auth_name_simons, auth_inst_simons, inst_name_simons, inst_num_simons, inst_auth_simons, num_auth_simons = author_institution_match(simons_auth, comma, simons_inst)
institution_data = np.load("data/institution_lat_long_simons_71.npz")
lat_long_simons  = institution_data['lat_long']

# Planck
auth_name_planck, auth_inst_planck, inst_name_planck, inst_num_planck, inst_auth_planck, num_auth_planck = author_institution_match(planck_auth, comma, planck_inst)
institution_data = np.load("data/institution_lat_long_planck_106.npz")
lat_long_planck  = institution_data['lat_long']

# Wmap
auth_name_wmap, auth_inst_wmap, inst_name_wmap, inst_num_wmap, inst_auth_wmap, num_auth_wmap = author_institution_match(wmap_auth, comma, wmap_inst)
institution_data = np.load("data/institution_lat_long_wmap_17.npz")
lat_long_wmap  = institution_data['lat_long']

# COBE
auth_name_cobe, auth_inst_cobe, inst_name_cobe, inst_num_cobe, inst_auth_cobe, num_auth_cobe = author_institution_match(cobe_auth, comma, cobe_inst)
institution_data = np.load("data/institution_lat_long_cobe_10.npz")
lat_long_cobe  = institution_data['lat_long']


In [138]:
# Create interactive folium map from instituton data, using different markers for each collaboration
def make_map(collaborations, inst_name_all, inst_auth_all, num_auth_all, lat_long_all):

    #set up colors and marker fills
    num_lists = len(inst_name_all)
    colors = ['#E37222', # tangerine
              '#0A8A9F', # teal
              '#FF1984', # pinkish 
              '#00B21E'] #green
    fills = [True] * num_lists

    

    # Folium create map object 
    lat_center, long_center = 35., -10. # stylistic choice 

    m = folium.Map(
        location=[lat_center, long_center],
        tiles='CartoDB dark_matter',
        zoom_start=2,
        min_zoom =2,
    )
    
    # Plot markers for each collaboration seperately
    for ll in range(num_lists):
        color = colors[ll]
        fill  = fills[ll]
        
        collaboration = collaborations[ll]
        inst_name = inst_name_all[ll]
        inst_auth = inst_auth_all[ll]
        num_auth  = num_auth_all[ll]

        lat_long = lat_long_all[ll]
        
        lat  = lat_long[:,0]
        long = lat_long[:,1]

        # Add markers as shapes, where radius is proportional to number of people^(1/2)
        feature_group = folium.FeatureGroup(name=collaboration)

        for i in range(len(lat)):
            # format popup text
            if num_auth[i] == 0: continue
            if num_auth[i] == 1:
                popup_text = "{}<br><br> {}<br><br> {} Member: {}<br>"
            else:
                popup_text = "{}<br><br> {}<br><br> {} Members: {}<br>"

            popup_text = popup_text.format(collaboration,
                                            inst_name[i],
                                            num_auth[i],
                                            inst_auth[i])
            
            # create circles
            if ll==0:                          
                folium.CircleMarker(
                    radius=4*np.sqrt(num_auth[i]),
                    location=[lat[i], long[i]],
                    popup=popup_text,
                    weight=2.,
                    color=color,
                    fill=fill,
                    fill_opacity=0.3, 
                ).add_to(feature_group)
            #create polygons
            else:
                folium.RegularPolygonMarker(
                    radius=4*np.sqrt(num_auth[i]),
                    number_of_sides=3+ll,
                    location=[lat[i], long[i]],
                    popup=popup_text,
                    weight=2.,
                    color=color,
                    fill_color=color,
                    fill_opacity=0.3,
                ).add_to(feature_group)
                
        feature_group.add_to(m)
    
    folium.LayerControl().add_to(m)
 

    # Create Legend with a bit of (messy) html
    legend_html_head = """ 
    <div style=
    "position: fixed;background-color: rgba(255, 255, 255, 0.7); border-radius: 5px;
     bottom: 50px; left: 10px; width: 175px; height:"""+str(37*num_lists)+"""px; border:2px solid grey; z-index:9999; font-size:14px;
     ">
     <p style="text-align:center;"> <b> Collaboration </b> </p>"""
    
    legend_html_tail="</div>"
    legend_html_body='' 

    for ll in range(num_lists):
        collaboration = collaborations[ll]
        color = colors[ll]
        legend_html_body+="""<p style="margin-left:5px"> <i class="fa fa-square fa-1x" style="margin-right:5px;color:"""+color+""";"></i>"""+collaboration+"""</p>"""

    legend_html = legend_html_head+legend_html_body+legend_html_tail
 
    # Adding Legend
    legend_layer = folium.Element(legend_html)
    m.get_root().html.add_child(legend_layer)

    return m

In [139]:
collaborations = ['Planck 2013 XVI', 'Simons Observatory', 'WMAP 9', 'COBE']
inst_name_plot = [inst_name_planck, inst_name_simons, inst_name_wmap, inst_name_cobe]
inst_auth_plot = [inst_auth_planck, inst_auth_simons, inst_auth_wmap, inst_auth_cobe]
num_auth_plot  = [num_auth_planck, num_auth_simons, num_auth_wmap, num_auth_cobe]
lat_long_plot  = [lat_long_planck, lat_long_simons, lat_long_wmap, lat_long_cobe]
m = make_map(collaborations, inst_name_plot, inst_auth_plot, num_auth_plot, lat_long_plot)


In [140]:
m

In [141]:
m.save("collaboration_visualizer_planck_simons_wmap_cobe.html")